# ----------------- Analyse Grid- und NetCDF-Daten mit R -----------------

## DGM einlesen und plotten

Als Einstieg schauen wir uns an, wie man ganz einfach einen schönen Plot eines DGMs mit R erstellen kann. Wir lesen das File "DG100_masked.tif" ein, extrahieren das Raster-Grid aus dem .tif Format und legen es als Raster-Objekt an (raster() aus der Bibliothek "raster"). Anschließend bestimmen wir mit der Funktion extent() die Eck-Koordinaten und Ausmaße und lassen uns diese ausgeben.

Mit NAvalue() weisen wir dem raster-Objekt "dem" dan Fehlwert zu, in diesem Fall 0.

Wir plotten das DGM mit der levelplot()-Funktion. Über den par.settings Parameter lassen sich verschieden Farbskalen einstellen, die wir per Variable "theme" auswählen und zuweisen. Hierzu benötigen wir die Bibliothek "rasterVis".

## Einlesen eines NetCDF-Files und Variablen extrahieren

Als nächstes wollen wir Daten eines regionalen Klimamodells (hier WRF) einlesen und damit arbeiten. Die Daten liegen im weit verbreiteten NetCDF-Format vor (https://www.unidata.ucar.edu/software/netcdf/ und https://de.wikipedia.org/wiki/NetCDF). Wir laden uns mit download.file() die Daten in unseren Workspace herunter. 

Mit der Funktion nc_open() aus der Bibliothek "ncdf4" lassen sich NetCDF-Dateien direkt in R öffnen. Wir lassen uns den Header, also die Metadaten des Files anzeigen, indem wir es aufrufen.

Als nächstes lesen wir mit ncvar_get() die Variablen XLONG und XLAT aus dem NetCDF-File aus und schreiben sie in die R-Variablen lon2d und lat2d und lassen den Anfang der Koordinaten-Felder ausgeben.

Wir lesen die Zeitvariable "Times" aus dem NetCDF-File und lassen uns diese ausgeben. So sehen wir, welche Zeitschritte in der Datei enthalten sind.

Nun schreiben wir die NetCDF-Variable "T2" mit dem Befehl ncvar_get() in unsere R-Variable "t2_array". Zusätzlichen holen wir uns die Attribute "description" und "units" mit ncatt_get() und zeigen diese an. Mit den Befehlen paste() und cat() lassen sich Text und Variableninhalte kombinieren und auf dem Bildschirm (hier im Notebook) ausgeben. Zusätzlich schauen wir uns die Dimensionen unseres 3-dimensionalen Arrays mit der Funktion dim() an. Die 3 Dimensionen des Arrays sind hier (wie sehr oft) die zwei Raumdimensionen x (246 Spalten) und y (285 Reihen) und die Zeitdimension t (30 Jahre * 12 Monate = 360 Zeitschritte).

## Berechnungen und Darstellung von mehrdimensionalen Feldern

Nun schneiden wir uns aus dem Grid-Stapel (=3d-Array =3d-Feld) einen Zeitschritt heraus, erzeugen also ein Grid am Zeitschritt i. Um die Dimensionen des Grids zu überprüfen benutzen wir wieder den Befehl dim().

Um das Grid zu plotten benötigen wir noch die Koordinaten lat und lon als eindimensionale Vektoren. Mit length() können wir uns deren Länge anzeigen lassen.

Das Projekt Colorbrewer erzeugt schöne Farbpaletten für verschiedene Anwendungen: http://colorbrewer2.org/. Diese vordefinierten Paletten lassen sich in fast jeder Programmiersprache über bereitgestellte Bibliotheken einbinden. Wir wollen diese hier benutzen mit der Bibliothek "RColorBrewer". Die Funktion display.brewer.all() stellt alle verfügbaren ColorBrewer-Paletten dar.

Nun plotten wir das Grid am Zeitschritt i mit der Funktion image.plot() aus der Bibliothek "fields" und einer von den obigen Farbpaletten und legen anschließend ein Grid darüber.

Das Grid können wir auch mit der levelplot()-Funktion darstellen. Hierzu müssen wir zunächst aus den zwei Vektoren lat und lon mit dem Befehl expand.grid() ein passendes Grid aufspannen.

Als nächstes wollen wir uns 4 Grids gleichzeitig in einem Panel-Plot anschauen. Hierzu schneiden wir uns 4 Grids aus dem 3d-Array an den Zeitschritten i1 bis i4 aus und plotten diese wieder mit image.plot(). 

Nun wollen wir die Temperaturdaten mit einem Zukunftsszenario vergleichen. In diesem Fall ist das ein regionales Klimamodell mit 5km horizontaler Auflösung, das mit dem Globalmodell MPI-ESM und dem Szenario RCP4.5 für den Zeitraum 2020-2049 angetrieben wurde. Wie oben laden wir uns das File herunter, öffnen das NetCDF-File und extrahieren die Variable T2.

Mit der Funktion apply(data,dims,fun) bilden wir den Mittelwert über alle Zeitschritte und subtrahieren den Wert 273.16 um von Kelvin auf °C zu kommen.

Wir erstellen jetzt zwei Plots dieser mittleren Lufttemperaturen:

Da die Temperaturunterschiede relativ gering sind können wir sie in der obigen Darstellung kaum erkennen. Wir bilden deshalb ein Differenzgrid, in dem wir die "Vergangenheit" von der "Zukunft" subtrahieren. Stimmen die Dimensionen der beiden Grids überein, können wir diese direkt voneinander abziehen.

... und plotten dieses zu unserem Plot von oben.

Wir stellen den Differenzplot noch einmal in groß und mit einer anderen Farbpalette dar (heat.colors()).

## Extrahieren einer Zeitreihe am Punkt x/y aus dem 3d-Array (Grid-Stapel) [x,y,t]

Oft möchte man aus einem 3d-Array (zwei Raumdimensionen, eine Zeitdimension) eine Zeitreihe an einem bestimmten Punkt extrahieren. Dies tun wir, indem wir eine neue Variable erstellen und diese durch Angabe der Dimensionen entsprechend belegen. Wir extrahieren also hier eine Zeitreihe an einem definierten Punkt (x/y) im Grid.

Wir prüfen kurz mit head() ob das funktioniert hat:

Um die Zeitreihe zu plotten erzeugen wir uns mit der Funktion seq() eine monatlichen Zeitachse...

... und plotten nun die Zeitreihe:

## Über Flächen gemittelte Zeitreihen

Nun bilden wir eine über eine bestimmte Fläche gemittelte Zeitreihe mit Hilfe der Funktion apply(). Wir machen dies für unsere Kontroll- und Szenarioperioden.

... und plotten die beiden Zeitreihen.

## Extrahieren einer Zeitreihe für bestimmte lat/lon-Koordinaten

Nun versuchen wir, aus dem 3D-Array die Zeitreihe an einer bestimmten geographischen Koordinate zu extrahieren, z.B. für Innsbruck (lon=11.404102 und lat=47.269212). Hierzu benutzen wir die Koordinaten aus den Vektoren lat und lon. Da die Gridpunkte evtl. nicht exakt diese Koordinaten reproduzieren, suchen wir mit der Funktion which() den Index des Gridpunkts, der am nächsten an den gesuchten Koordinaten liegt. Dies machen wir mit einer Abfrage des minimalen Abstands z.B wie folgt: ind_lon=which(abs(lon-11.404102)==min(abs(lon-11.404102)))

Für einen Vergleich erzeugen wir nun einen Panel-Plot für die Temperaturzeitreihen des Flächenmittels, der Temperatur am Punkt x/y und der Temperatur in Innsbruck.

## Ausschneiden einer Box aus dem ursprünglichen Grid(stapel)

Abschließend wollen wir noch eine bestimmte rechteckige Region aus dem Differenzen-Grid ausschneiden und darstellen. Wir fokussieren beispielsweise auf den Alpenraum mit den Koordinaten: lon_min=6, lon_max=13.5, lat_min=44, lat_max=48. Wir benutzen dazu wieder die Funktion which(), die uns mit einer konditionellen Abfrage die richtigen Indizes im Grid liefert.

Wir stellen die Region mit levelplot() dar und benötigen dafür wieder die entsprechenden lat und lon Vektoren.